How to select a bounding box ( ROI ) in OpenCV (C++/Python):  
https://www.learnopencv.com/how-to-select-a-bounding-box-roi-in-opencv-cpp-python/  

Convert opencv image to greyscale:  
https://techtutorialsx.com/2018/06/02/python-opencv-converting-an-image-to-gray-scale/  

Inverting opencv image:  
https://stackoverflow.com/questions/19580102/inverting-image-in-python-with-opencv  

Find all bounding boxes (didn't work):  
https://stackoverflow.com/questions/21104664/extract-all-bounding-boxes-using-opencv-python  

Bounding boxes, pyimagesearch.com:  
https://www.pyimagesearch.com/2014/04/21/building-pokedex-python-finding-game-boy-screen-step-4-6/  


In [1]:
# maybe deskew the image?
# draw bounding boxes
# get contours
# slice up boxes

In [2]:
from PIL import Image
from pathlib import Path
from itertools import tee
from os import path
import cv2
import numpy as np
import imutils

In [3]:
# not used yet...
class CroppingTemplate:
    def __init__(self):
        self.image_file = None  # posix path
        self.img = None
        self.counter = 1
        self.dest_dir = "./major_pieces"
        
        # these are the box boundaries for each section
        self.major_sections = [
            {"A": {
                "left": blank,
                "top": 700,
                "right": 12560,
                "bottom": 1900}
            }
        ]

    def load(self):
        self.img = Image.open(str(self.image_file))

    def crop_all2(self):
        for section in self.major_sections:
            for dir_name, box in section.items():
                cropped_img = self.img.crop((box["left"], box["top"], box["right"], box["bottom"]))
                file_name = path.join(self.dest_dir, dir_name, str(self.counter)+".jpg")
                cropped_img.save(file_name)
        print("Finished: ", self.counter)
        self.counter += 1

In [4]:
# test_file = "./major_pieces/uppercase/1.jpg"
test_file = "small_uppercase.jpg"

# Read image
img = cv2.imread(test_file)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# bilateralFilter reduces noise while preserving boundaries
gray = cv2.bilateralFilter(gray, 11, 17, 17)

# invert the image for making bounding boxes
inverted = cv2.bitwise_not(gray)

# get edges
# edged = cv2.Canny(gray, 30, 200)
edged = cv2.Canny(inverted, 30, 200)

In [5]:
# find contours in the edged image, keep only the largest
# ones, and initialize our screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
screenCnt = None

In [6]:
# loop over our contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.015 * peri, True)
    # if our approximated contour has four points, then
    # we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

In [ ]:
# cv2.drawContours(img, screenCnt, -1, (0, 255, 0), 3)
cv2.drawContours(img, cnts, -1, (0, 255, 0), 1)
cv2.imshow("Game Boy Screen", img)
cv2.waitKey(0)